# L-Layer Testing
## Architecture

```json
{
    "epochs": 1,
    "layers": 2,
    "activations": {
        "layer1": "sigmoid",
        "layer2": "relu"
    },
    "neurons": {
        "layer1": 3,
        "layer2": 1
    },
    "weight": 0.01,
    "bias": 0,
    "learning_rate": 0.005
}
```

---

---
## Forward Propogation
### S3 Event

In [2]:
context = ''
# Simulate S3 event trigger data
event = {
    "Records": [
        {
            "eventVersion": "2.0",
            "eventTime": "1970-01-01T00:00:00.000Z",
            "requestParameters": {
                "sourceIPAddress": "127.0.0.1"
             },
            "s3": {
                "configurationId": "testConfigRule",
                "object": {
                    "eTag": "0123456789abcdef0123456789abcdef",
                    "sequencer": "0A1B2C3D4E5F678901",
                    "key": "training_input/datasets.h5",
                    "size": 1024
                },
                "bucket": {
                    "arn": "arn:aws:s3:::lnn",
                    "name": "lnn",
                    "ownerIdentity": {
                        "principalId": "EXAMPLE"
                    }
                },
                "s3SchemaVersion": "1.0"
            },
            "responseElements": {
                "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH",
                "x-amz-request-id": "EXAMPLE123456789"
            },
            "awsRegion": "us-west-2",
            "eventName": "ObjectCreated:Put",
            "userIdentity": {
                "principalId": "EXAMPLE"
            },
            "eventSource": "aws:s3"
        }
    ]
}

---
### Launch

In [3]:
import launch
from launch import *
launch.lambda_handler(event, context)
np.random.seed(1)

Complete Neural Network Settings: 

{
    "activations": {
        "layer1": "relu",
        "layer2": "sigmoid"
    },
    "bias": 0,
    "data_keys": {
        "W1": "W1|float64#3#12288",
        "W2": "W2|float64#1#3",
        "b1": "b1|float64#3#1",
        "b2": "b2|float64#1#1",
        "results": "results|json",
        "test_set_x": "test_set_x|float64#12288#50",
        "test_set_y": "test_set_y|int64#1#50",
        "train_set_x": "train_set_x|float64#12288#209",
        "train_set_y": "train_set_y|int64#1#209"
    },
    "dims": {
        "test_set_x": [
            12288,
            50
        ],
        "test_set_y": [
            1,
            50
        ],
        "train_set_x": [
            12288,
            209
        ],
        "train_set_y": [
            1,
            209
        ]
    },
    "epochs": 1,
    "input_data": [
        "train_set_x",
        "train_set_y",
        "test_set_x",
        "test_set_y"
    ],
    "layers": 2,
    "learning_rate": 0.00

---
### Launch -> Trainer

In [4]:
event = {
    "parameter_key": "parameters|json",
    "state": "start"
}

In [5]:
import trainer
from trainer import *
trainer.lambda_handler(event, context)

Starting Forward Propogation for epoch 0, layer 1
Payload to be sent NeuronLambda: 
{
    "activation": "relu",
    "epoch": 0,
    "id": 1,
    "last": "False",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Payload to be sent NeuronLambda: 
{
    "activation": "relu",
    "epoch": 0,
    "id": 2,
    "last": "False",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Payload to be sent NeuronLambda: 
{
    "activation": "relu",
    "epoch": 0,
    "id": 3,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
### Trainer -> Neuron (Layer 1)

In [6]:
import neuron
from neuron import *

event = {
    "1": {
        "activation": "relu",
        "epoch": 0,
        "id": 1,
        "last": "False",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    },
    "2": {
        "activation": "relu",
        "epoch": 0,
        "id": 2,
        "last": "False",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    },
    "3": {
        "activation": "relu",
        "epoch": 0,
        "id": 3,
        "last": "True",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    }
}
event = dumps(event)
event = loads(event)

In [7]:
layer = 3
parameters = from_cache(endpoint=endpoint, key="parameters|json")
if layer == parameters['layers'] + 1:
    print("True")

True


In [8]:
parameter_key = event.get('1')['parameter_key']
parameters = from_cache(endpoint=endpoint, key=parameter_key)
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 1,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

In [9]:
layer = event.get('1')['layer']
print("Processing Layer: " + str(layer))
event1 = event.get('1')
print("Neuron 1 event:\n" + dumps(event1))
neuron.lambda_handler(event=event1, context='')
event2 = event.get('2')
print("Neuron 2 event:\n" + dumps(event2))
neuron.lambda_handler(event=event2, context='')
event3 = event.get('3')
neuron.lambda_handler(event=event3, context='')
print("Neuron 3 event:\n" + dumps(event3))

Processing Layer: 1
Neuron 1 event:
{"activation": "relu", "epoch": 0, "id": 1, "last": "False", "layer": 1, "parameter_key": "parameters|json", "state": "forward"}
Neuron 2 event:
{"activation": "relu", "epoch": 0, "id": 2, "last": "False", "layer": 1, "parameter_key": "parameters|json", "state": "forward"}
Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Neuron 3 event:
{"activation": "relu", "epoch": 0, "id": 3, "last": "True", "layer": 1, "parameter_key": "parameters|json", "state": "forward"}


---
### Neuron (Layer 1) -> Trainer

In [10]:
import trainer
from trainer import *
event = {
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer.lambda_handler(event, context)

Propogating forward onto Layer 2
Starting Forward Propogation for epoch 0, layer 2
Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}


**Sanity Check: Confirm that `TrainerLambda` processed Layer 1 Activations**

In [11]:
parameters = from_cache(endpoint=endpoint, key='parameters|json')
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'A1': 'A1|float64#3#209',
  'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 2,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

In [12]:
A1 = from_cache(endpoint=endpoint, key=parameters['data_keys']['A1'])
A1.shape

(3, 209)

---
### Trainer -> Neuron (Layer 2)

In [13]:
import neuron
from neuron import *
event = {
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
neuron.lambda_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 3,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
### Neuron (Layer 2) -> Trainer

In [14]:
import trainer
from trainer import *
event = {
    "epoch": 0,
    "layer": 3,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer.lambda_handler(event, context)

Cost after epoch 0: 0.6931378502927884
Starting Backward Propogation for epoch 0, layer 2
Payload to be sent to NeuronLambda: 
{
    "activastion": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "backward"
}


**Sanity Check: Confirm `results` have been updated with the correct cost.**

In [15]:
parameters = from_cache(endpoint=endpoint, key="parameters|json")
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'A1': 'A1|float64#3#209',
  'A2': 'A2|float64#1#209',
  'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'dA2': 'dA2|float64#1#209',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 2,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

In [16]:
A2 = from_cache(endpoint=endpoint, key=parameters['data_keys']['A2'])
A2.shape

(1, 209)

In [17]:
tmp = from_cache(endpoint=endpoint, key=parameters['data_keys']['results'])
print("Current results output:\n")
tmp

Current results output:



{'epoch0': {'cost': 0.6931378502927884}}

---

---
## Back Propogation
### Trainer -> Neuron (Layer 2)
** Sanity Check: Confirm that the derivative of the Cost Function (`dA2`) is initialized.**

In [18]:
parameters = from_cache(endpoint=endpoint, key="parameters|json")
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'A1': 'A1|float64#3#209',
  'A2': 'A2|float64#1#209',
  'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'dA2': 'dA2|float64#1#209',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 2,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

>**Note:** The derivative of the Cost is already built within the `TrainerLambda` and the event already sent to the `NeuronLambda`. Therefore the next step in the workflow is to process the derivatives for Layer 2 on the Neuron.

---
### Neuron (Layer 2) -> Trainer

In [19]:
import neuron
from neuron import *
event = {
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "backward"
}
#neuron.lambda_handler(event, context)

In [20]:
def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [21]:
def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [22]:
layer = 2
ID = 1
if layer == parameters['layers']:
    da = from_cache(
        endpoint=endpoint,
        key=parameters['data_keys']['dA'+str(layer)])[ID-1, :].reshape(
            1,
            parameters['dims']['train_set_y'][1]
        )
    a = from_cache(
        endpoint=endpoint,
        key=parameters['data_keys']['A'+str(layer)])[ID-1, :].reshape(
            1,
            parameters['dims']['train_set_y'][1]
        )
print("da shape: " + str(da.shape))
print("a shape: " + str(a.shape))

da shape: (1, 209)
a shape: (1, 209)


>**Note:** Do I need to hard code the shape? If so, does it need to be based on the layer OR does that `if` statment happen for `A0`?

In [23]:
layer = 2
ID = 1
da = from_cache(
    endpoint=endpoint,
    key=parameters['data_keys']['dA'+str(layer)]
)
da.shape

(1, 209)

>**Note:** After further thought, the Neuron needs to be idempotent to potential multi-classification or CovNet etc., so I need to hard code the shape because if I don't then the shape will be `(209,)`. But do I still need to hard code this to the last layer?

In [24]:
layer = 2
ID = 1
da = from_cache(
    endpoint=endpoint,
    key=parameters['data_keys']['dA'+str(layer)])[ID-1, :].reshape(
        parameters['neurons']['layer'+str(layer)],
        parameters['dims']['train_set_y'][1]
)
da.shape

(1, 209)

In [25]:
a = from_cache(
    endpoint=endpoint,
    key=parameters['data_keys']['A'+str(layer)])[ID-1, :].reshape(
        parameters['neurons']['layer'+str(layer)],
        parameters['dims']['train_set_y'][1]
)
a.shape

(1, 209)

In [26]:
# Derivative of the non-linear activation
activation = event.get('activation')
print("activation: " + activation)
if activation == "sigmoid":
    dz = sigmoid_backward(da, a)
elif activation == "relu":
    dz = relu_backward(da, a)
else:
    print("Backward Activation function not yet iplemented")
dz.shape

activation: sigmoid


(1, 209)

In [27]:
da

array([[ 2.00044257,  2.00052776, -2.        ,  1.99955604,  2.00081408,
         2.00090029,  1.99964713, -2.        ,  2.        ,  2.        ,
         2.        , -2.00013402,  2.        , -2.00044353, -1.99981028,
         2.00036471,  2.00045343,  2.00089246,  2.        , -2.00011096,
         2.        ,  1.99978175,  1.99999013,  2.00025545, -2.00064185,
        -2.00040766,  1.99914226, -2.00091657,  1.99976457, -1.99947795,
         2.00004083,  2.        ,  2.00016709,  2.        ,  1.99940015,
         2.        ,  1.99962628,  2.        , -2.00023868,  2.        ,
         2.00013184, -2.00069642, -1.99989408,  2.00010593,  2.        ,
         2.0006029 ,  2.00056452, -1.99987177,  1.99932981,  2.        ,
        -2.        ,  2.        ,  2.        ,  2.        , -1.99963586,
         2.00027624, -2.        , -2.        ,  2.00002596, -2.00008488,
        -2.        , -2.        ,  2.        ,  1.99857634,  2.        ,
         1.99982322,  2.        ,  2.        , -1.9

In [28]:
dz

array([[ 0.47009869,  0.47011626, -0.47000742,  0.46991587,  0.4701753 ,
         0.47019308,  0.46993465, -0.47000742,  0.47000742,  0.47000742,
         0.47000742, -0.47004278,  0.47000742, -0.47012442, -0.46995738,
         0.47008264,  0.47010093,  0.47019147,  0.47000742, -0.47003669,
         0.47000742,  0.46996242,  0.47000539,  0.47006011, -0.47017673,
        -0.47011496,  0.46983053, -0.47024919,  0.46995887, -0.46986971,
         0.47001584,  0.47000742,  0.47004188,  0.47000742,  0.46988372,
         0.47000742,  0.46993035,  0.47000742, -0.47007038,  0.47000742,
         0.47003461, -0.47019112, -0.46997949,  0.47002927,  0.47000742,
         0.47013176,  0.47012384, -0.4699736 ,  0.46986921,  0.47000742,
        -0.47000742,  0.47000742,  0.47000742,  0.47000742, -0.46991137,
         0.47006439, -0.47000742, -0.47000742,  0.47001278, -0.47002981,
        -0.47000742, -0.47000742,  0.47000742,  0.46971382,  0.47000742,
         0.46997097,  0.47000742,  0.47000742, -0.4

In [ ]:
# Get the Forward parameters for W, b and and activations for layer -1
